<a href="https://colab.research.google.com/github/SakshiGehani/J015_NLPassignments/blob/HomeWork/HW11_QnA_J015_17_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

--2020-02-22 17:19:14--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.110.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json.4’

train-v2.0.json.4   100%[===================>]  40.17M   182MB/s    in 0.2s    

2020-02-22 17:19:14 (182 MB/s) - ‘train-v2.0.json.4’ saved [42123633/42123633]



In [0]:
import json
import pandas as pd

In [3]:
df = pd.read_json('/content/train-v2.0.json')
df.head()

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...


In [4]:
df = df.drop(['version'], 1)
df.head()

,data
0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{..."
1,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,{'title': 'Sino-Tibetan_relations_during_the_M...
3,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu..."
4,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...


In [5]:
df.data[0]

{'paragraphs': [{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
   'qas': [{'answers': [{'answer_start': 269, 'text': 'in the late 1990s'}],
     'id': '56be85543aeaaa14008c9063',
     'is_impossible': False,
     'question': 'When did Beyonce start becoming popular?'},
    {'answers': [{'answer_start': 207, 'text': 'singing and danc

In [0]:
with open("/content/train-v2.0.json", "r") as f:
    data = json.loads(f.read())

ques = []
ans = []

for i in range(0, len(data["data"])):
    for j in range(0, len(data["data"][i]["paragraphs"])):
        for k in range(len(data["data"][i]["paragraphs"][j]["qas"])):
            q = data["data"][i]["paragraphs"][j]["qas"][k]["question"]
            try: 
                a = data["data"][i]["paragraphs"][j]["qas"][k]["answers"][0]["text"]
            except IndexError:
                a = "None"

            ques.append(q)
            ans.append(a)

data_dict = {"Questions": ques, "Answers": ans}

In [0]:
df1 = pd.DataFrame(data_dict)

In [8]:
df1.head()

,Questions,Answers
0,When did Beyonce start becoming popular?,in the late 1990s
1,What areas did Beyonce compete in when she was...,singing and dancing
2,When did Beyonce leave Destiny's Child and bec...,2003
3,In what city and state did Beyonce grow up?,"Houston, Texas"
4,In which decade did Beyonce become famous?,late 1990s


In [9]:
# importing required modules
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('words')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [0]:
## Preprocessing data

stop_words = set(stopwords.words('english'))      # for unique entries -> set()
words = set(nltk.corpus.words.words())

def preprocess(sen):
  sen = sen.lower()
  no_punc = sen.translate(sen.maketrans('', '', '!"$%&\'()#@*+,-./:;<=>?[\\]^_`{|}~'))      ## removing all punctuation marks
  filtered = no_punc.translate(no_punc.maketrans('', '', '#@0123456789'))           ## removing digits
  filtered = re.sub(r'[^\w\s]', ' ', filtered)            ## removing non-Unicode character strings
  tokens = word_tokenize(filtered)                        ## creating a list of tokens
  no_stop_words = [w for w in tokens if not w in stop_words]     # eliminate stop words;  returns list of lists

  sent = " ".join(w for w in no_stop_words)
  return sent

In [11]:
corpus = df1['Questions'].apply(preprocess)
corpus 

0                          beyonce start becoming popular
1                           areas beyonce compete growing
2         beyonce leave destinys child become solo singer
3                                 city state beyonce grow
4                            decade beyonce become famous
                               ...                       
130314                  physics broadly agreed definition
130315                        coined term partonic matter
130316                            another name antimatter
130317               matter usually need used conjunction
130318               field study variety unusual contexts
Name: Questions, Length: 130319, dtype: object

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 2000)

X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())

['abbey', 'ability', 'able', 'ac', 'academic', 'academy', 'accept', 'accepted', 'access', 'according', 'account', 'accounts', 'accused', 'achieve', 'acronym', 'across', 'act', 'action', 'actions', 'active', 'activity', 'acts', 'actually', 'ad', 'add', 'added', 'addition', 'address', 'administration', 'adopt', 'adopted', 'adult', 'advantage', 'affect', 'affected', 'affirmative', 'afl', 'africa', 'african', 'age', 'agencies', 'agency', 'ages', 'ago', 'agree', 'agreed', 'agreement', 'agriculture', 'aid', 'air', 'aircraft', 'airport', 'alaska', 'albert', 'album', 'alcohol', 'alexander', 'alliance', 'allow', 'allowed', 'allows', 'almost', 'along', 'alps', 'also', 'alternative', 'always', 'amendment', 'america', 'american', 'americans', 'americas', 'among', 'amount', 'analysis', 'ancestry', 'ancient', 'animal', 'animals', 'ann', 'annelids', 'announce', 'announced', 'annual', 'another', 'antarctica', 'antenna', 'anthropology', 'antibiotics', 'appear', 'appeared', 'apple', 'applications', 'app

In [13]:
#converting the count vectorizer array into input dataframe
X = pd.DataFrame(X.toarray(), columns=(vectorizer.get_feature_names()))
X.head()

,abbey,ability,able,ac,academic,academy,accept,accepted,access,according,account,accounts,accused,achieve,acronym,across,act,action,actions,active,activity,acts,actually,ad,add,added,addition,address,administration,adopt,adopted,adult,advantage,affect,affected,affirmative,afl,africa,african,age,...,whose,widely,william,win,windows,winter,within,without,witnesses,woman,women,wood,word,words,work,worked,workers,working,works,world,worlds,worldwide,worst,worth,would,write,writer,writers,writing,written,wrote,yale,year,years,york,young,youtube,zhejiang,zinc,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
#bag_of_words = corpus.tolist()
bag_of_words = []
for i in range(corpus.shape[0]):
  for word in corpus[i].split():
    bag_of_words.append(word)
bag_of_words = list(set(bag_of_words))

In [15]:
print(bag_of_words)

['aerodrome', 'ricardos', 'michelin', 'balkans', 'dubstep', 'four', 'pickups', 'permian', 'ecuador', 'pavla', 'ewell', 'drawbacks', 'lollard', 'inconvenient', 'sumitoma', 'raa', 'chymotrypsin', 'moldavia', 'sands', 'agrarian', 'militarism', 'recommend', 'ramon', 'deicing', 'attests', 'romancespeaking', 'hải', 'height', 'allcolor', 'saintétienne', 'lagged', 'psychologists', 'inflected', 'machination', 'investigatory', 'jayanti', 'transupport', 'reconfirm', 'treasurer', 'fivebit', 'italia', 'monikered', 'macroorganism', 'tintin', 'nullification', 'conference', 'says', 'myohoji', 'jars', 'bourgeoisie', 'instantly', 'kamil', 'pablo', 'jamaicas', 'daytona', 'teflon', 'biotechnology', 'gnss', 'classics', 'century', 'form', 'nbc', 'collections', 'tres', 'harmonizing', 'verrazzanos', 'uninspired', 'stratus', 'peta', 'emphysema', 'rothblatt', 'chord', 'bible', 'navys', 'kasmir', 'category', 'confessed', 'tritium', 'beja', 'episkyros', 'shut', 'rpm', 'coarser', 'projects', 'assimilating', 'guade

In [16]:
len(bag_of_words)

45120

In [17]:
import gensim
from gensim.models import Word2Vec
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
w2v_model = Word2Vec(list(bag_of_words), size=50000, min_count=1, sg=1, workers=5, window=3)

In [0]:
sentence1 = preprocess(df1['Questions'][0])
sentence2 = preprocess(df1['Questions'][1])
sim_score = w2v_model.wv.similarity(sentence1.lower().split(), sentence2.lower().split())